<a href="https://colab.research.google.com/github/HaJunYoo/Pyspark-tutorial/blob/main/category_review_average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Spark/01-spark'

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar -xvf spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark
!pip install pyspark

spark-3.3.1-bin-hadoop2/
spark-3.3.1-bin-hadoop2/LICENSE
spark-3.3.1-bin-hadoop2/NOTICE
spark-3.3.1-bin-hadoop2/R/
spark-3.3.1-bin-hadoop2/R/lib/
spark-3.3.1-bin-hadoop2/R/lib/SparkR/
spark-3.3.1-bin-hadoop2/R/lib/SparkR/DESCRIPTION
spark-3.3.1-bin-hadoop2/R/lib/SparkR/INDEX
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/Rd.rds
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/features.rds
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/hsearch.rds
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/links.rds
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/nsInfo.rds
spark-3.3.1-bin-hadoop2/R/lib/SparkR/Meta/package.rds
spark-3.3.1-bin-hadoop2/R/lib/SparkR/NAMESPACE
spark-3.3.1-bin-hadoop2/R/lib/SparkR/R/
spark-3.3.1-bin-hadoop2/R/lib/SparkR/R/SparkR
spark-3.3.1-bin-hadoop2/R/lib/SparkR/R/SparkR.rdb
spark-3.3.1-bin-hadoop2/R/lib/SparkR/R/SparkR.rdx
spark-3.3.1-bin-hadoop2/R/lib/SparkR/help/
spark-3.3.1-bin-hadoop2/R/lib/SparkR/help/AnIndex
spark-3.3.1-bin-hadoop2/R/lib/SparkR/h

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark3_test").master("local[*]").getOrCreate()

import pyspark

In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
conf = SparkConf().setMaster("local").setAppName("category-review-average")
sc = SparkContext.getOrCreate(conf=conf)

In [ ]:
directory = path + "/data"
filename = "restaurant_reviews.csv"

In [ ]:
lines = sc.textFile(f"file:///{directory}/{filename}")
# line RDD를 만듦

In [ ]:
lines.collect()

['id,item,cateogry,reviews',
 '0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [ ]:
header = lines.first()

In [ ]:
filtered_lines = lines.filter(lambda row: row != header)
# row들을 가져오는데 header만 빼고 다 가져오기 

In [ ]:
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [ ]:
def parse(row):
    #'0,짜장면,중식,125'
    fields = row.split(",")
    # 하나의 row를 받아와서 parsing함
    category = fields[2]
    reviews = int(fields[3])
    return (category, reviews)

In [ ]:
categoryReviews = filtered_lines.map(parse)
# key value RDD가 만들어지게 된다

In [ ]:
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [ ]:
categoryReviewsCount = categoryReviews.mapValues(lambda x: (x, 1))
# 각 카테고리 당 값 하나씩 추가 

In [ ]:
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [ ]:
reduced = categoryReviewsCount.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
# 합산 해서 keyvalue rdd 생성 

In [ ]:
reduced.collect()

[('분식', (566, 2)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2)),
 ('중식', (360, 2)),
 ('일식', (287, 3))]

In [ ]:
averages = reduced.mapValues(lambda x: x[0] / x[1])
averages.collect()

[('분식', 283.0),
 ('아시안', 312.0),
 ('패스트푸드', 17.5),
 ('중식', 180.0),
 ('일식', 95.66666666666667)]